In [1]:
import polars as pl
from polars import col as c
import altair as alt
from camminapy.plot import altair_theme

altair_theme()

In [2]:
df = pl.read_parquet("../data/data.parquet")

In [3]:
df

Rank,Mark,Competitor,DOB,Nat,Pos,Venue,Date,Results Score,Event,Wind,Sex,Event Type,Event Unit,Event Distance,On Track,On Road,Is Running,Is Walking,Performance,Relative Mark
i64,f64,str,date,str,str,str,date,i64,str,f64,str,str,str,i64,bool,bool,bool,bool,str,f64
1,223.1,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Stadio Olimpic…",1999-07-07,1292,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.0
2,223.4,"""Noah NGENY""",1978-11-02,"""KEN""","""2""","""Stadio Olimpic…",1999-07-07,1288,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.001345
3,224.3,"""Noureddine MOR…",1970-02-28,"""ALG""","""1""","""Rieti (ITA)""",1993-09-05,1275,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.005379
4,224.6,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Nice (FRA)""",1998-07-16,1272,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.006723
5,224.9,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Oslo (NOR)""",1997-07-04,1268,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.008068
6,224.9,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Stadio Olimpic…",2001-06-29,1267,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.008068
7,225.1,"""Noureddine MOR…",1970-02-28,"""ALG""","""1""","""Zürich (SUI)""",1995-08-16,1264,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.008965
8,225.6,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Olympiastadion…",1997-08-26,1258,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.011206
9,225.9,"""Hicham EL GUER…",1974-09-14,"""MAR""","""1""","""Crystal Palace…",2000-08-05,1254,"""One Mile""",null,"""male""","""Middle Distanc…","""Metres""",1609,true,false,true,false,"""lower is bette…",1.01255


In [26]:
(
    alt.Chart(
        df.sort("Rank")
        .groupby("Event", "Sex", maintain_order=True)
        .head(20)
        .drop("Date", "DOB")
        .filter(c("Is Running"))
        .filter(c("On Track"))
        .filter(c("Event").str.contains("Hurdles").is_not())
        .filter(c("Event").str.contains("Steeple").is_not())
        .sort("Event Distance")
    )
    .mark_line(interpolate="step", clip=True, point=True, strokeWidth=2)
    .encode(
        x=alt.X("Rank:N").scale(zero=False),
        y=alt.Y("Relative Mark:Q").scale(domain=(1, 1.03)),
        color=(
            alt.Color("Event:N", sort=None).legend(
                columns=2, symbolLimit=0, labelLimit=0
            )
        ),
        # strokeDash="Sex:N",
        row="Sex:N",
        tooltip=["Event:N"],
    )
    .properties(width=1400, height=400)
    .interactive()
)

alt.Chart(...)